简介: 利用重新定义网络的方法来测试训练好的model

# 准备工作

In [ ]:
import tensorflow as tf
import numpy as np
import Ipynb_importer
import myNet_infer
import myNet_train

读入训练数据

In [ ]:
"""读入，待测试的数据"""
mnist = myNet_train.read_date();\
input_x = np.reshape(mnist.train.images, (mnist.train.num_examples, 28, 28, 1))
label = mnist.train.labels

定义数据集评价方法

In [ ]:
numTest = mnist.train.num_examples
x = tf.placeholder(tf.float32, shape=[100, 28, 28 ,1], name="x_input")
y_label = tf.placeholder(tf.float32, shape = [100, 10], name="y_label")

y_pred = myNet_infer.LeNet_inference(x);

correct_prediction = tf.equal(tf.argmax(y_label, 1), tf.argmax(y_pred, 1))
accuracy = tf.reduce_sum(tf.cast(correct_prediction, tf.int32)) 


# 在训练集上 评价训练好的模型(验证是否high bias)

In [ ]:
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.7  # 程序最多只能占用指定gpu70%的显存
config.gpu_options.allow_growth = True 
all_correct = 0
with tf.Session(config=config) as sess:
    saver = tf.train.Saver()
    #saver = tf.train.import_meta_graph('./model_param/model.ckpt-1001.meta') #加载图"""
    #saver.restore(sess, "./model_param/model.ckpt-1001") # 恢复遍历, 注意这里没有文件后缀
    ckpt = tf.train.get_checkpoint_state("model_param/")
    print("name of used model: ", ckpt.model_checkpoint_path)
    print("\n")
    if ckpt and ckpt.model_checkpoint_path:
        # load model
        saver.restore(sess, ckpt.model_checkpoint_path)
        for i in range(595):
            num_correct = sess.run(accuracy,feed_dict={x:input_x[i*100:(i+1)*100,:,:,:], y_label:label[i*100:(i+1)*100, :]}) 
            all_correct += num_correct
        print(all_correct, all_correct/mnist.train.num_examples)
    else:
        print("no model or can not find model")

# 在测试集上, 评价模型(验证是否high variance)

In [ ]:
test_x = np.reshape(mnist.test.images, (mnist.test.num_examples, 28, 28, 1))
test_label = mnist.test.labels

all_correct = 0
with tf.Session(config=config) as sess:
    saver = tf.train.Saver()
    #saver = tf.train.import_meta_graph('./model_param/model.ckpt-1001.meta') #加载图"""
    #saver.restore(sess, "./model_param/model.ckpt-1001") # 恢复遍历, 注意这里没有文件后缀
    ckpt = tf.train.get_checkpoint_state("model_param_1/")
    print("name of used model", ckpt.model_checkpoint_path)
    if ckpt and ckpt.model_checkpoint_path:
        # load model
        saver.restore(sess, ckpt.model_checkpoint_path)
        for i in range(100):
            num_correct = sess.run(accuracy,feed_dict={x:test_x[i*100:(i+1)*100,:,:,:], y_label:test_label[i*100:(i+1)*100, :]}) 
            all_correct += num_correct
        print(all_correct, all_correct/mnist.test.num_examples)
    else:
        print("no model or can not find model", )